In [4]:
# import necessary packages

import pandas as pd # for working with tables
pd.options.mode.chained_assignment = None  # default = 'warn'

import os # for finding file directory with os.getcwd()

# for calculations
import numpy as np
import math

import time

import matplotlib.pyplot as plt # for plotting

# for plotting histogram with percentages
from matplotlib.ticker import PercentFormatter
import matplotlib.ticker as mtick

from PIL import Image # for converting plots to black-and-white

from ipynb.fs.full.MyFunctions_00 import *

## Create Colors Dataframes

In [6]:
# read in the files for ALLWISE matches from void and wall galaxies from SDSS
# older files used 10" matching radius, while new files use 6"
voids_6arc = read_data('IPAC_Results/New/Voids/new_voids_AWSource')
walls_6arc = read_data('IPAC_Results/New/Walls/new_walls_AWSource')

voids_6arc = voids_6arc.rename(columns = {'nsaid_01' : 'object_tag'})
walls_6arc = walls_6arc.rename(columns = {'nsaid_01' : 'object_tag'})

voids_6arc.head()

read IPAC_Results/New/Voids/new_voids_AWSource
read IPAC_Results/New/Walls/new_walls_AWSource


,cntr_01,dist_x,pang_x,object_tag,ra_01,dec_01,ra,dec,w1mpro,w1sigmpro,w1snr,w2mpro,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr
0,1,0.151786,98.322261,52,146.594503,-0.133150,146.594545,-0.133156,15.346,0.040,27.2,15.241,0.095,11.5,11.979,NaN,1.2,8.623,NaN,0.8
1,2,2.434494,-81.609628,75,145.874464,-0.608775,145.873795,-0.608677,14.144,0.043,25.2,14.034,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5
2,3,0.173687,-28.459110,95,146.222266,-0.375920,146.222243,-0.375877,13.238,0.025,44.0,13.075,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0
3,4,0.036896,45.446259,121,145.681110,-0.867233,145.681118,-0.867226,13.040,0.024,46.2,13.006,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5
4,5,2.015983,83.997238,127,145.759730,-0.813891,145.760287,-0.813832,13.355,0.051,21.4,13.228,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4


In [7]:
# filter tables from walls and voids for both matching radii

print(len(voids_6arc))
voids_filtered = filter_colors(voids_6arc)
print(len(voids_filtered))

print("")

print(len(walls_6arc))
walls_filtered = filter_colors(walls_6arc)
print(len(walls_filtered))

86891
84676

245778
242531


In [5]:
voids_filtered.head()

,cntr_01,dist_x,pang_x,object_tag,ra_01,dec_01,ra,dec,w1mpro,w1sigmpro,w1snr,w2mpro,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr
0,1,0.151786,98.322261,52,146.594503,-0.133150,146.594545,-0.133156,15.346,0.040,27.2,15.241,0.095,11.5,11.979,NaN,1.2,8.623,NaN,0.8
1,2,2.434494,-81.609628,75,145.874464,-0.608775,145.873795,-0.608677,14.144,0.043,25.2,14.034,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5
2,3,0.173687,-28.459110,95,146.222266,-0.375920,146.222243,-0.375877,13.238,0.025,44.0,13.075,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0
3,4,0.036896,45.446259,121,145.681110,-0.867233,145.681118,-0.867226,13.040,0.024,46.2,13.006,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5
4,5,2.015983,83.997238,127,145.759730,-0.813891,145.760287,-0.813832,13.355,0.051,21.4,13.228,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4


In [7]:
# for both groups, calcuate a new column containing two colors for each galaxy
# calculate the mean and standard deviation of both colors for later plotting
voids_colors = voids_filtered

voids_colors['W1-W2'] = voids_filtered.w1mpro - voids_filtered.w2mpro
voids_colors['W2-W3'] = voids_filtered.w2mpro - voids_filtered.w3mpro
voids_colors = voids_colors.rename(columns = {"object_tag_01": "object_tag"})

walls_colors = walls_filtered

walls_colors['W1-W2'] = walls_filtered.w1mpro - walls_filtered.w2mpro
walls_colors['W2-W3'] = walls_filtered.w2mpro - walls_filtered.w3mpro
walls_colors = walls_colors.rename(columns = {"object_tag_01": "object_tag"})

In [5]:
print(walls_colors.columns)
walls_colors.head()

Index(['cntr_01', 'dist_x', 'pang_x', 'object_tag', 'ra_01', 'dec_01', 'ra',
       'dec', 'w1mpro', 'w1sigmpro', 'w1snr', 'w2mpro', 'w2sigmpro', 'w2snr',
       'w3mpro', 'w3sigmpro', 'w3snr', 'w4mpro', 'w4sigmpro', 'w4snr', 'W1-W2',
       'W2-W3'],
      dtype='object')


,cntr_01,dist_x,pang_x,object_tag,ra_01,dec_01,ra,dec,w1mpro,w1sigmpro,...,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr,W1-W2,W2-W3
0,1,0.621310,-88.040692,464,148.728855,0.188863,148.728682,0.188869,14.783,0.035,...,0.057,19.2,10.944,0.128,8.5,8.427,0.377,2.9,0.301,3.538
1,2,0.067083,-153.002538,570,147.428682,1.107487,147.428674,1.107470,13.279,0.024,...,0.028,39.3,11.477,0.211,5.1,8.779,NaN,0.4,0.142,1.660
2,3,0.147374,74.127786,572,147.511343,1.100596,147.511382,1.100608,14.043,0.028,...,0.042,25.9,11.941,NaN,0.8,9.017,NaN,-0.5,0.068,2.034
3,4,0.024719,-59.380482,576,147.561502,1.229841,147.561496,1.229845,13.677,0.026,...,0.032,33.5,10.517,0.086,12.6,7.995,NaN,1.8,0.178,2.982
4,5,0.052153,-119.381587,615,148.015821,1.148428,148.015808,1.148420,13.925,0.026,...,0.038,28.8,12.296,NaN,1.0,9.303,NaN,-0.8,0.018,1.611


In [6]:
print(voids_colors.columns)
voids_colors.head()

Index(['cntr_01', 'dist_x', 'pang_x', 'object_tag', 'ra_01', 'dec_01', 'ra',
       'dec', 'w1mpro', 'w1sigmpro', 'w1snr', 'w2mpro', 'w2sigmpro', 'w2snr',
       'w3mpro', 'w3sigmpro', 'w3snr', 'w4mpro', 'w4sigmpro', 'w4snr', 'W1-W2',
       'W2-W3'],
      dtype='object')


,cntr_01,dist_x,pang_x,object_tag,ra_01,dec_01,ra,dec,w1mpro,w1sigmpro,...,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr,W1-W2,W2-W3
0,1,0.151786,98.322261,52,146.594503,-0.133150,146.594545,-0.133156,15.346,0.040,...,0.095,11.5,11.979,NaN,1.2,8.623,NaN,0.8,0.105,3.262
1,2,2.434494,-81.609628,75,145.874464,-0.608775,145.873795,-0.608677,14.144,0.043,...,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5,0.110,3.232
2,3,0.173687,-28.459110,95,146.222266,-0.375920,146.222243,-0.375877,13.238,0.025,...,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0,0.163,3.465
3,4,0.036896,45.446259,121,145.681110,-0.867233,145.681118,-0.867226,13.040,0.024,...,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5,0.034,1.967
4,5,2.015983,83.997238,127,145.759730,-0.813891,145.760287,-0.813832,13.355,0.051,...,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4,0.127,2.721


## Merge Colors into Groups

In [23]:
# read in saved Pr tables
voids_parent = read_data_db('voids_parent_' + 'new_' + '04-11-23') # timestamp())
walls_parent = read_data_db('walls_parent_' + 'new_' + '04-11-23') # timestamp())

voids_bright = read_data_db('voids_bright_' + 'new_' + '04-11-23') # timestamp())
walls_bright = read_data_db('walls_bright_' + 'new_' + '04-11-23') # timestamp())

voids_main = read_data_db('voids_main_' + 'new_' + '04-11-23') # timestamp())
walls_main = read_data_db('walls_main_' + 'new_' + '04-11-23') # timestamp())

voids_dwarf = read_data_db('voids_dwarf_' + 'new_' + '04-11-23') # timestamp())
walls_dwarf = read_data_db('walls_dwarf_' + 'new_' + '04-11-23') # timestamp())

read voids_parent_new_04-11-23
read walls_parent_new_04-11-23
read voids_bright_new_04-11-23
read walls_bright_new_04-11-23
read voids_main_new_04-11-23
read walls_main_new_04-11-23
read voids_dwarf_new_04-11-23
read walls_dwarf_new_04-11-23


In [9]:
voids_parent.head(10)

,object_tag,num_obj,Pr,vflag,RA,DEC,PLATE,FIBERID,MJD,rabsmag_NSA,Z,SFR,sSFR,BPTclass,HImass,u_r,g_r,u_r_err,g_r_err
0,75.0,24.0,0.933297,1,145.874464,-0.608775,266,138,51630,-20.979137,0.071590,0.437294,-10.228045,2.0,NaN,1.570181,0.527512,0.092720,0.031002
1,95.0,24.0,0.948802,1,146.222266,-0.375920,266,168,51630,-20.544321,0.067406,0.436883,-10.211826,1.0,NaN,1.867657,0.645546,0.077289,0.028860
2,121.0,23.0,0.987329,1,145.681110,-0.867233,266,207,51630,-21.178394,0.067595,-1.141768,-12.134328,5.0,NaN,2.711828,0.698425,0.282711,0.029654
3,127.0,22.0,0.998411,1,145.759730,-0.813891,266,216,51630,-20.736423,0.067634,-0.348571,-11.222754,4.0,NaN,2.237722,0.712046,0.085222,0.028771
4,128.0,22.0,0.900567,1,145.708927,-0.747680,266,217,51630,-19.849358,0.071566,-1.348428,-11.774809,-1.0,NaN,2.189016,0.687010,0.130119,0.031340
5,171.0,20.0,0.534975,1,145.256954,-1.101891,266,287,51630,-19.169422,0.066550,NaN,NaN,NaN,NaN,1.838703,0.599663,0.190723,0.036573
6,196.0,23.0,0.630624,1,145.296063,0.801639,266,339,51630,-20.109673,0.079322,-1.425182,-12.124483,-1.0,NaN,2.198677,0.792656,0.181813,0.033932
7,206.0,15.0,0.034938,1,145.288565,0.346527,266,359,51630,-19.407125,0.064601,0.037810,-9.946843,1.0,NaN,1.088249,0.424688,0.102724,0.040913
8,208.0,25.0,-0.398992,1,145.868108,0.829419,266,362,51630,-20.248100,0.082600,-0.539767,-11.249261,5.0,NaN,2.413645,0.732395,0.203511,0.031472
9,210.0,24.0,0.877024,1,145.544641,0.939367,266,369,51630,-20.530415,0.083907,-0.942053,-11.839916,-1.0,NaN,2.423882,0.831264,0.210281,0.032650


In [10]:
print(walls_bright.head(10)) 

   object_tag  num_obj        Pr  vflag          RA       DEC  PLATE  FIBERID  \
0       464.0     21.0  0.765606      0  148.728855  0.188863    267      152   
1       570.0     23.0  0.980080      0  147.428682  1.107487    267      362   
2       572.0     21.0  0.788920      0  147.511343  1.100596    267      369   
3       576.0     23.0  0.801471      0  147.561502  1.229841    267      378   
4       644.0     22.0  0.464834      0  148.236848  0.611061    267      482   
5       646.0     23.0  0.755275      0  148.317834  0.666678    267      484   
6       648.0     23.0  0.464870      0  148.290262  0.583504    267      486   
7       656.0     24.0  0.622570      0  148.311885  0.649937    267      499   
8       657.0     21.0  0.868472      0  148.307956  0.628529    267      500   
9       660.0     20.0  0.851046      0  148.429697  0.666487    267      507   

     MJD  rabsmag_NSA         Z       SFR       sSFR  BPTclass  HImass  \
0  51608   -20.542107  0.094445  0

In [11]:
print(voids_bright.head())

   object_tag  num_obj        Pr  vflag          RA       DEC  PLATE  FIBERID  \
0        75.0     24.0  0.933297      1  145.874464 -0.608775    266      138   
1        95.0     24.0  0.948802      1  146.222266 -0.375920    266      168   
2       121.0     23.0  0.987329      1  145.681110 -0.867233    266      207   
3       127.0     22.0  0.998411      1  145.759730 -0.813891    266      216   
4       196.0     23.0  0.630624      1  145.296063  0.801639    266      339   

     MJD  rabsmag_NSA         Z       SFR       sSFR  BPTclass  HImass  \
0  51630   -20.979137  0.071590  0.437294 -10.228045       2.0     NaN   
1  51630   -20.544321  0.067406  0.436883 -10.211826       1.0     NaN   
2  51630   -21.178394  0.067595 -1.141768 -12.134328       5.0     NaN   
3  51630   -20.736423  0.067634 -0.348571 -11.222754       4.0     NaN   
4  51630   -20.109673  0.079322 -1.425182 -12.124483      -1.0     NaN   

        u_r       g_r   u_r_err   g_r_err  
0  1.570181  0.527512  0

In [12]:
voids_parent_merged = voids_parent.merge(voids_colors, how = 'left', on = 'object_tag')
voids_parent_merged.head(20)

,object_tag,num_obj,Pr,vflag,RA,DEC,PLATE,FIBERID,MJD,rabsmag_NSA,...,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr,W1-W2,W2-W3
0,75.0,24.0,0.933297,1,145.874464,-0.608775,266,138,51630,-20.979137,...,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5,0.110,3.232
1,95.0,24.0,0.948802,1,146.222266,-0.375920,266,168,51630,-20.544321,...,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0,0.163,3.465
2,121.0,23.0,0.987329,1,145.681110,-0.867233,266,207,51630,-21.178394,...,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5,0.034,1.967
3,127.0,22.0,0.998411,1,145.759730,-0.813891,266,216,51630,-20.736423,...,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4,0.127,2.721
4,128.0,22.0,0.900567,1,145.708927,-0.747680,266,217,51630,-19.849358,...,0.041,26.8,12.431,NaN,0.0,8.759,NaN,-0.6,0.166,1.533
5,171.0,20.0,0.534975,1,145.256954,-1.101891,266,287,51630,-19.169422,...,0.052,20.7,11.152,0.148,7.4,8.671,0.329,3.3,0.202,3.373
6,196.0,23.0,0.630624,1,145.296063,0.801639,266,339,51630,-20.109673,...,0.035,30.7,11.651,NaN,1.8,8.791,NaN,0.4,0.109,2.199
7,206.0,15.0,0.034938,1,145.288565,0.346527,266,359,51630,-19.407125,...,0.056,19.5,11.170,0.143,7.6,8.489,NaN,1.8,0.195,3.524
8,208.0,25.0,-0.398992,1,145.868108,0.829419,266,362,51630,-20.248100,...,0.038,28.9,11.208,0.163,6.6,8.246,NaN,1.1,0.131,2.486
9,210.0,24.0,0.877024,1,145.544641,0.939367,266,369,51630,-20.530415,...,0.035,31.0,12.207,0.423,2.6,8.824,NaN,-0.5,0.138,1.334


In [13]:
voids_bright_merged = voids_bright.merge(voids_colors, how = 'left', on = 'object_tag')
voids_bright_merged.head(20)

,object_tag,num_obj,Pr,vflag,RA,DEC,PLATE,FIBERID,MJD,rabsmag_NSA,...,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr,W1-W2,W2-W3
0,75.0,24.0,0.933297,1,145.874464,-0.608775,266,138,51630,-20.979137,...,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5,0.110,3.232
1,95.0,24.0,0.948802,1,146.222266,-0.375920,266,168,51630,-20.544321,...,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0,0.163,3.465
2,121.0,23.0,0.987329,1,145.681110,-0.867233,266,207,51630,-21.178394,...,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5,0.034,1.967
3,127.0,22.0,0.998411,1,145.759730,-0.813891,266,216,51630,-20.736423,...,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4,0.127,2.721
4,196.0,23.0,0.630624,1,145.296063,0.801639,266,339,51630,-20.109673,...,0.035,30.7,11.651,NaN,1.8,8.791,NaN,0.4,0.109,2.199
5,208.0,25.0,-0.398992,1,145.868108,0.829419,266,362,51630,-20.248100,...,0.038,28.9,11.208,0.163,6.6,8.246,NaN,1.1,0.131,2.486
6,210.0,24.0,0.877024,1,145.544641,0.939367,266,369,51630,-20.530415,...,0.035,31.0,12.207,0.423,2.6,8.824,NaN,-0.5,0.138,1.334
7,326.0,22.0,0.073209,1,146.782326,0.794416,266,578,51630,-20.130468,...,0.037,29.5,12.520,NaN,-0.2,8.942,NaN,-0.6,0.058,1.154
8,348.0,23.0,0.133700,1,147.294003,0.381281,266,624,51630,-20.413872,...,0.085,12.8,11.362,0.268,4.0,7.932,NaN,0.6,0.261,2.914
9,351.0,24.0,0.994995,1,147.270863,0.468847,266,630,51630,-21.514750,...,0.023,46.9,11.763,0.329,3.3,8.701,NaN,0.7,-0.030,0.467


In [25]:
voids_parent_nona = merge_nona(voids_parent, voids_colors)
walls_parent_nona = merge_nona(walls_parent, walls_colors)

voids_bright_nona = merge_nona(voids_bright, voids_colors)
walls_bright_nona = merge_nona(walls_bright, walls_colors)

voids_main_nona = merge_nona(voids_main, voids_colors)
walls_main_nona = merge_nona(walls_main, walls_colors)

voids_dwarf_nona = merge_nona(voids_dwarf, voids_colors)
walls_dwarf_nona = merge_nona(walls_dwarf, walls_colors)

fraction lost = 0.001555
fraction lost = 0.001448
fraction lost = 0.001325
fraction lost = 0.001365
fraction lost = 0.001737
fraction lost = 0.001545
fraction lost = 0.001393
fraction lost = 0.001852


In [15]:
print(len(voids_bright) + len(voids_main) + len(voids_dwarf))
print(len(voids_bright_nona) + len(voids_main_nona) + len(voids_dwarf_nona))

71372
71261


In [16]:
print(len(voids_parent))
print(len(voids_parent_nona))

71372
71261


In [17]:
print(len(walls_bright) + len(walls_main) + len(walls_dwarf))
print(len(walls_bright_nona) + len(walls_main_nona) + len(walls_dwarf_nona))

219573
219255


In [18]:
print(len(walls_parent))
print(len(walls_parent_nona))

219573
219255


In [19]:
# voids_parent_nona.to_csv('Files/voids_parent_colors_' + 'new_' + timestamp() + '.csv', index = False)
# walls_parent_nona.to_csv('Files/walls_parent_colors_' + 'new_' + timestamp() + '.csv', index = False)

# voids_bright_nona.to_csv('Files/voids_bright_colors_' + 'new_' + timestamp() + '.csv', index = False)
# walls_bright_nona.to_csv('Files/walls_bright_colors_' + 'new_' + timestamp() + '.csv', index = False)

# voids_main_nona.to_csv('Files/voids_main_colors_' + 'new_' + timestamp() + '.csv', index = False)
# walls_main_nona.to_csv('Files/walls_main_colors_' + 'new_' + timestamp() + '.csv', index = False)

# voids_dwarf_nona.to_csv('Files/voids_dwarf_colors_' + 'new_' + timestamp() + '.csv', index = False)
# walls_dwarf_nona.to_csv('Files/walls_dwarf_colors_' + 'new_' + timestamp() + '.csv', index = False)

# Check increase in voids

In [26]:
print(len(voids_parent_nona))
voids_parent_nona.head()

71261


,object_tag,num_obj,Pr,vflag,RA,DEC,PLATE,FIBERID,MJD,rabsmag_NSA,...,w2sigmpro,w2snr,w3mpro,w3sigmpro,w3snr,w4mpro,w4sigmpro,w4snr,W1-W2,W2-W3
0,75.0,24.0,0.933297,1,145.874464,-0.608775,266,138,51630,-20.979137,...,0.057,19.1,10.802,0.124,8.7,8.286,0.435,2.5,0.110,3.232
1,95.0,24.0,0.948802,1,146.222266,-0.375920,266,168,51630,-20.544321,...,0.029,37.9,9.610,0.044,24.9,8.194,0.273,4.0,0.163,3.465
2,121.0,23.0,0.987329,1,145.681110,-0.867233,266,207,51630,-21.178394,...,0.027,39.9,11.039,0.139,7.8,8.913,NaN,-0.5,0.034,1.967
3,127.0,22.0,0.998411,1,145.759730,-0.813891,266,216,51630,-20.736423,...,0.058,18.7,10.507,0.120,9.1,7.438,NaN,1.4,0.127,2.721
4,128.0,22.0,0.900567,1,145.708927,-0.747680,266,217,51630,-19.849358,...,0.041,26.8,12.431,NaN,0.0,8.759,NaN,-0.6,0.166,1.533


In [27]:
print(len(voids_parent_nona))
voids_parent_nona = voids_parent_nona.dropna(subset = ['W1-W2'])
print(len(voids_parent_nona))

71261
71261


In [28]:
print(len(voids_parent_nona))
voids_parent_nona_w3 = voids_parent_nona.dropna(subset = ['W2-W3'])
print(len(voids_parent_nona_w3))

71261
71259
